Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [1]:
import retinoto_py as fovea
args = fovea.Params()
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, num_epochs=5, n_train_stop=0, seed=1998, batch_size=250, model_name='resnet50', do_scratch=False, shuffle=True, verbose=True)

# testing a ResNet50 model on the validation dataset

In [2]:
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'

In [3]:
from retinoto_py import get_loader
val_loader, class_to_idx, idx_to_class = get_loader(args, VAL_DATA_DIR)

In [4]:
from retinoto_py import load_model, count_parameters, count_layers
model = load_model(args)
model.eval()
# --- Parameter Counts ---
param_stats = count_parameters(model)
print("🔢 Parameter Count:")
print(f"  Total:     {param_stats['total_parameters']:,}")
print(f"  Trainable: {param_stats['trainable_parameters']:,}")
print("-" * 50)

# --- Layer Counts ---
print("🧱 Layer Count:")
# 1. Total number of nn.Module objects
total_layers = count_layers(model)
print(f"  Total Modules (nn.Module): {total_layers}")

# 2. Counting specific layer types
from torch import nn

conv_layers = count_layers(model, layer_type=nn.Conv2d)
linear_layers = count_layers(model, layer_type=nn.Linear)
print(f"  Convolutional (nn.Conv2d): {conv_layers}")
print(f"  Linear (nn.Linear): {linear_layers}")
print("=" * 50)

🔢 Parameter Count:
  Total:     25,557,032
  Trainable: 25,557,032
--------------------------------------------------
🧱 Layer Count:
  Total Modules (nn.Module): 151
  Convolutional (nn.Conv2d): 53
  Linear (nn.Linear): 1


In [5]:
import torch
from tqdm.auto import tqdm
import pandas as pd

def get_validation_accuracy(args, model, val_loader):
    model.eval()

    correct_predictions = 0
    total_predictions = 0

    for images, true_labels in tqdm(val_loader, desc=f"Evaluating {args.model_name}"):
        images = images.to(args.device)
        true_labels = true_labels.to(args.device)

        # Get predictions (no need for gradients)
        with torch.no_grad():
            outputs = model(images)
            _, predicted_labels = torch.max(outputs, dim=1)

        # --- Check for Correctness ---
        # The batch size is 1, so we get the first element of the tensors
        predicted_label_idx = predicted_labels[0].item()
        true_label_idx = true_labels[0].item()

        # Check if the prediction was correct for the entire batch
        # The comparison produces a tensor of booleans (True/False)
        correct_predictions_in_batch = (predicted_labels == true_labels)

        # Sum the boolean tensor to get the number of correct predictions in the batch
        # .item() extracts the number from the tensor
        correct_predictions += correct_predictions_in_batch.sum().item()

        # The total number of predictions is the batch size
        total_predictions += true_labels.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy

json_filename = args.data_cache / '11_model_accuracy.json'

if json_filename.exists():
    results = pd.read_json(json_filename)
else:
    accuracy = get_validation_accuracy(args, model, val_loader)
    results = pd.DataFrame({'model_name':args.model_name, 'accuracy': accuracy}, index=[0])
    results.to_json(json_filename, orient='index', indent=2)

print(f"Evaluation complete.")
print(f"Accuracy: {results[0]['accuracy']*100:.3f}%")

Evaluation complete.
Accuracy: 80.724%


# testing the role of batch_size on Wall Clock Time 

In [ ]:
import time

json_filename = args.data_cache / '11_batch_size.json'

if json_filename.exists():
    results = pd.read_json(json_filename)
else:
    args = fovea.Params()
    accuracy = get_validation_accuracy(args, model, val_loader)
    results = pd.DataFrame({'model_name':args.model_name, 'accuracy': accuracy}, index=[0])
    results.to_json(json_filename, orient='index', indent=2)


if json_filename.exists():
    results = pd.read_json(json_filename)
else:
    all_results = []
    for batch_size in [2, 4, 8, 16, 32, 64, 128, 256]:
        args = fovea.Params(batch_size=batch_size)
        tic = time.time()
        model = load_model(args)
        accuracy = get_validation_accuracy(args, model, val_loader)
        toc = time.time()
        this_result = {         
                        'batch_size': batch_size,
                        'accuracy': accuracy,
                        'wall_clock_time': toc-tic,
                        }
        all_results.append(this_result)
    results = pd.DataFrame(all_results)
    results.to_json(json_filename, orient='index', indent=2)



Evaluating resnet50:   0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Create the Plot ---
sns.set_theme(style="whitegrid", font_scale=1.2) # Set a nice, readable theme

plt.figure(figsize=(10, 6)) # Set the figure size

# Use lineplot with markers to show data points clearly
ax = sns.lineplot(
    data=results,
    x='batch_size',          # Batch size on the x-axis
    y='wall_clock_time',     # Wall clock time on the y-axis
    marker='o',              # Add a circle marker for each data point
    markersize=8,            # Make the markers a bit larger
    lw=2                     # Set a nice line width
)

# --- Final Touches ---
# Improve axis labels and title for clarity
ax.set_xlabel('Batch Size')
ax.set_ylabel('Wall Clock Time (seconds)')
ax.set_title('Model Inference Time vs. Batch Size')

# Ensure the x-axis shows all batch sizes as ticks, not just a few
ax.set_xticks(results['batch_size'])

# Optional: Add a secondary y-axis to show how accuracy changes
ax2 = ax.twinx()
ax2 = sns.lineplot(
    data=results, 
    x='batch_size', 
    y='accuracy', 
    ax=ax2, 
    color='orange', 
    marker='D', 
    markersize=8, 
    alpha=0.7,
    lw=2,
    label='Accuracy'
)
ax2.set_ylabel('Accuracy')
ax2.grid(False) # Hide the grid for the secondary axis to avoid clutter

# Get handles and labels for the legend
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax.legend(h1+h2, l1+l2, loc='upper left')


plt.tight_layout()
plt.show()


In [ ]:
args.num_workers

# testing different networks

In [ ]:
args = fovea.Params()
from retinoto_py import all_models

json_filename = args.data_cache / '11_model_comparison.json'

if json_filename.exists():
    results = pd.read_json(json_filename)
else:
    all_results = []
    for model_name in all_models:
        tic = time.time()
        args = fovea.Params(model_name=model_name)
        model = load_model(args)
        param_stats = count_parameters(model)
        total_layers = count_layers(model)
        accuracy = get_validation_accuracy(args, model, val_loader)
        toc = time.time()
        this_result = {'model_name': model_name,
                                'accuracy': accuracy,
                                'wall_clock_time': toc-tic,
                                'total_parameters': param_stats['total_parameters'],
                                'trainable_parameters': param_stats['trainable_parameters'],
                                'total_layers': total_layers}
        all_results.append(this_result)
    results = pd.DataFrame(all_results)
    results.to_json(json_filename, orient='index', indent=2)


In [ ]:


# --- Step 1: Reshape the DataFrame from Wide to Long ---
# This makes plotting with seaborn much easier.
id_vars = ['model_name', 'accuracy'] # Columns to keep as-is
value_vars = ['wall_clock_time', 'total_parameters', 'total_layers'] # Columns to unpivot

results_long = pd.melt(
    results,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='metric', # Name for the new column that will hold the metric names
    value_name='value'  # Name for the new column that will hold the metric values
)

# Convert value columns to more readable names for plot titles
metric_name_map = {
    'wall_clock_time': 'Wall Clock Time (s)',
    'total_parameters': 'Total Parameters (M)',
    # 'total_layers': 'Total Layers'
}
results_long['metric_label'] = results_long['metric'].map(metric_name_map)


# --- Step 2: Create the Multi-Panel Plot ---
sns.set_theme(style="whitegrid", font_scale=1.1) # Set a nice theme

# Create a FacetGrid
# This will create a separate plot (a "facet") for each unique value in the 'metric_label' column
g = sns.FacetGrid(
    results_long,
    col="metric_label",    # Create a panel for each metric
    hue="model_name",      # Color lines by model
    col_wrap=32,            # Arrange panels in 2 columns
    height=4,              # Height of each panel
    aspect=1.2             # Width of each panel (aspect * height)
)

# Map a lineplot onto the grid
g.map_dataframe(
    sns.lineplot,
    x="accuracy",
    y="value",
    marker="o",           # Add a marker to each data point
    lw=2                  # Line width
)

# --- Step 3: Final Touches ---
# g.map() returns the grid object, so we can now customize it.
g.set_titles("{col_name}") # Set each panel title to the metric name
g.set_axis_labels("Accuracy", "Value") # Set common axis labels

# Improve the legend
g.add_legend(title="Model", bbox_to_anchor=(1.05, 0.9), loc='upper left') # Move legend outside

# Adjust layout to prevent title/legend overlap
plt.tight_layout()

# Show the plot
plt.show()